# Calculate intercoder reliability

In [1]:
import pandas as pd
import numpy as np
import simpledorff

In [6]:
#annotations data
df = pd.read_csv('../annotations_131_overlap_round_2.csv.csv')

In [7]:
df

,coder_id,coder,jobset,unit_id,unit_status,variable,value,field
0,2605,monika@annotator.com,All,3287472_3290620,DONE,topic_match,Yes,NaN
1,2605,monika@annotator.com,All,3287472_3290620,DONE,confidence_1,++,NaN
2,2605,monika@annotator.com,All,3287472_3290620,DONE,event_match,No,NaN
3,2605,monika@annotator.com,All,3287472_3290620,DONE,confidence_2,++,NaN
4,2607,roeland_d@annotator.com,All,3287472_3290620,DONE,topic_match,Yes,NaN
...,...,...,...,...,...,...,...,...
1419,2608,floris_v@annotator.com,All,3287550_6290523,DONE,confidence_2,++,NaN
1420,2608,floris_v@annotator.com,All,3287500_3290580,DONE,topic_match,Yes,NaN
1421,2608,floris_v@annotator.com,All,3287500_3290580,DONE,confidence_1,++,NaN
1422,2608,floris_v@annotator.com,All,3287500_3290580,DONE,event_match,No,NaN


In [50]:
#overlap set
d = pd.read_csv('../overlap_data.csv')

In [3]:
# Pivot the DataFrame to calculate Krippendorff's alpha
pivoted_df = df.pivot(index=['coder_id', 'coder', 'jobset', 'unit_id', 'unit_status'],
                     columns='variable', values='value').reset_index()

pivoted_df = pivoted_df.rename_axis(None, axis=1)

# If there are NaN values in the pivoted DataFrame, you can replace them with a default value like this
pivoted_df = pivoted_df.fillna('N/A')

In [194]:
# exclude rows with unit_id containing 3285386 (kruiswoordtest)
excluded_df = pivoted_df[~pivoted_df['unit_id'].str.contains('3285386')]

In [4]:
#calculate alpha before any removal of flagged cases
alpha_topic = simpledorff.calculate_krippendorffs_alpha_for_df(pivoted_df,
                                                        experiment_col='unit_id',
                                                        annotator_col='coder_id',
                                                        class_col='topic_match')
alpha_topic

0.7825051055139551

In [5]:
#calculate alpha before any removal of flagged cases
alpha_event = simpledorff.calculate_krippendorffs_alpha_for_df(pivoted_df,
                                                        experiment_col='unit_id',
                                                        annotator_col='coder_id',
                                                        class_col='event_match')
alpha_event

0.7534722222222222

In [8]:
disagreement_table = pd.crosstab(pivoted_df['coder_id'], pivoted_df['topic_match'])
print(disagreement_table)


topic_match  No  Yes
coder_id            
2605         29   60
2606         33   56
2607         34   55
2608         34   55


In [9]:
disagreement_table = pd.crosstab(pivoted_df['coder_id'], pivoted_df['event_match'])
print(disagreement_table)

event_match  No  Yes
coder_id            
2605         81    8
2606         77   12
2607         79   10
2608         83    6


In [22]:
disagreement_table = pd.crosstab(index=[pivoted_df['unit_id'], pivoted_df['coder_id']], columns=pivoted_df['topic_match'], margins=True, margins_name="Total")
disagreement_table


topic_match               No  Not sure/Flag  Yes  Total
unit_id         coder_id                               
3285233_3285386 2592       0              1    0      1
                2602       0              1    0      1
                2603       0              1    0      1
                2604       1              0    0      1
3285240_6290579 2592       0              0    1      1
...                       ..            ...  ...    ...
3290705_3290570 2592       0              0    1      1
                2602       0              0    1      1
                2603       0              0    1      1
                2604       0              0    1      1
Total                     97             15  244    356

[357 rows x 4 columns]

In [147]:
# Assuming 'pivoted_df' and 'd' are your DataFrames
# Replace 'pivoted_df' and 'd' with the actual names of your DataFrames

# Merge the DataFrames on 'unit_id' in pivoted_df and 'id' in d
merged_df = pd.merge(pivoted_df, d, left_on='unit_id', right_on='id', how='inner')

# The 'how' parameter specifies the type of merge. You can use 'inner', 'left', 'right', or 'outer' as needed.
#merged_df

In [137]:
# exclude rows with unit_id containing 3285386 (kruiswoordtest)
excluded_df = merged_df[~merged_df['unit_id'].str.contains('3285386')]

In [149]:
#calculate alpha before any removal of flagged cases
alpha_topic = simpledorff.calculate_krippendorffs_alpha_for_df(excluded_df,
                                                        experiment_col='unit_id',
                                                        annotator_col='coder_id',
                                                        class_col='topic_match')
alpha_topic

0.6634674727627684

In [153]:
#calculate alpha before any removal of flagged cases
alpha_event = simpledorff.calculate_krippendorffs_alpha_for_df(excluded_df,
                                                        experiment_col='unit_id',
                                                        annotator_col='coder_id',
                                                        class_col='event_match')
alpha_event

0.40635437881873715

### For Main topic level

In [154]:
# Assuming that 'merged_df' is a Pandas DataFrame
filtered_df = merged_df[merged_df['topic_match'] == 'Not sure/Flag']

In [172]:
#get all cases where flag/not sure was answered
practice_list = filtered_df['unit_id'].unique()

In [186]:
# Filter the DataFrame 'd' to retain only rows where 'unit_id' matches values in 'unit_ids'
#to check what we all coded initially on these cases
initial_df = merged_df[merged_df['unit_id'].isin(practice_list)]

In [189]:
# List of columns to drop
columns_to_drop = ['Publisher1', 'Publisher2', 'ID1', 'ID2']

# Drop the specified columns
initial_df.drop(columns=columns_to_drop, inplace=True)

/tmp/ipykernel_2904255/480310955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_df.drop(columns=columns_to_drop, inplace=True)


In [193]:
disagreement_table = pd.crosstab(index=[initial_df['unit_id'], initial_df['coder_id']], columns=initial_df['topic_match'], margins=True)
disagreement_table.to_excel('../test.xlsx')

In [173]:
# Filter the DataFrame 'd' to retain only rows where 'unit_id' matches values in 'unit_ids'
practice_df = d[d['id'].isin(practice_list)]

In [179]:
len(practice_df)
practice_df.to_csv('../practice.csv')
practice_df.to_excel('../practice.xlsx')

### For News events level

In [202]:
# Assuming that 'merged_df' is a Pandas DataFrame
filtered_df = merged_df[merged_df['event_match'] == 'Not sure/Flag']
#get all cases where flag/not sure was answered
practice_list = filtered_df['unit_id'].unique()
# Filter the DataFrame 'd' to retain only rows where 'unit_id' matches values in 'unit_ids'
#to check what we all coded initially on these cases
initial_df = merged_df[merged_df['unit_id'].isin(practice_list)]
# List of columns to drop
columns_to_drop = ['Publisher1', 'Publisher2', 'ID1', 'ID2']

# Drop the specified columns
initial_df.drop(columns=columns_to_drop, inplace=True)

/tmp/ipykernel_2904255/3078711920.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_df.drop(columns=columns_to_drop, inplace=True)


In [203]:
disagreement_table = pd.crosstab(index=[initial_df['unit_id'], initial_df['coder_id']], columns=initial_df['event_match'], margins=True)
disagreement_table.to_excel('../test2.xlsx')

In [204]:
disagreement_table

event_match               No  Not sure/Flag  Yes  All
unit_id         coder_id                             
3285233_3285386 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3285335_6290568 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3285637_3285386 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3286186_3285370 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3287516_3286373 2592       0              0    1    1
                2602       1              0    0    1
                2603       0              0    1    1
                2604       0              1    0    1
3287527_3285252 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3287534_3285370 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3287538_6290568 2592       1              0    0    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       1              0    0    1
3290580_6290568 2592       0              0    1    1
                2602       1              0    0    1
                2603       0              1    0    1
                2604       0              0    1    1
All                       23              9    4   36

In [206]:
# Filter the DataFrame 'd' to retain only rows where 'unit_id' matches values in 'unit_ids'
practice_df = d[d['id'].isin(practice_list)]
len(practice_df)

9

In [207]:
practice_df.to_csv('../practice_2.csv')
practice_df.to_excel('../practice_2.xlsx')

In [161]:
grouped_df = filtered_df.groupby('unit_id')
grouped_df

group = grouped_df.get_group('3285233_3285386')

In [162]:
group

,coder_id,coder,jobset,unit_id,unit_status,confidence_1,confidence_2,event_match,topic_match,Unnamed: 0,...,Date2,Publisher1,Publisher2,ID1,ID2,proper_nouns1,proper_nouns2,keywords1,keywords2,id
0,2592,m.simon@uva.nl,All,3285233_3285386,DONE,+/-,++,No,Not sure/Flag,2,...,2021-03-01 00:00:00,De Volkskrant,Trouw,3285233,3285386,"Baudet, VERSLAGGEVERSCOLUMN, Baudet, Forum voo...","Roger Federer voor miljoenen, SP, VVD, 18coupl...","['parsifal', 'lockdown', 'ingeruild', 'verslag...","['europeanen', 'geactualiseerde', 'anagr', 'wi...",3285233_3285386
1,2602,roeland@annotator.com,All,3285233_3285386,DONE,+/-,++,No,Not sure/Flag,2,...,2021-03-01 00:00:00,De Volkskrant,Trouw,3285233,3285386,"Baudet, VERSLAGGEVERSCOLUMN, Baudet, Forum voo...","Roger Federer voor miljoenen, SP, VVD, 18coupl...","['parsifal', 'lockdown', 'ingeruild', 'verslag...","['europeanen', 'geactualiseerde', 'anagr', 'wi...",3285233_3285386
2,2603,sin-ching@annotator.com,All,3285233_3285386,DONE,++,+/-,Not sure/Flag,Not sure/Flag,2,...,2021-03-01 00:00:00,De Volkskrant,Trouw,3285233,3285386,"Baudet, VERSLAGGEVERSCOLUMN, Baudet, Forum voo...","Roger Federer voor miljoenen, SP, VVD, 18coupl...","['parsifal', 'lockdown', 'ingeruild', 'verslag...","['europeanen', 'geactualiseerde', 'anagr', 'wi...",3285233_3285386


In [163]:
group['Text1'][0]

"In Haarlem Alle politici zijn standwerkers, maar Thierry Baudet houdt dus ineens van Nederlandstalige rap; VERSLAGGEVERSCOLUMN Thierry Baudet draagt een pet en houdt van Nederlandstalige rap sommige dingen veranderen sneller en onverwachter dan de winter in de lente. Zijn Latijn is ingeruild voor de taal van rapper Duncan, die in het voorprogramma staat van alweer een event ondanks de coronaregels houdt Forum voor Democratie tweemaal daags een verkiezingsoptreden bij u in de buurt, dat bij de betreffende autoriteiten is aangemeld als demonstratie. Duncan rapt Niemand trekt z'n bek open ik doe dit voor de mensen die opstaan en het durven te zeggen er is hier veel te veel mis maar niemand wilt 't beseffen. Thierry Baudet, de ooit over z'n vleugel gevouwen politicus die de prelude van Parsifal prefereerde en een afschuw had van het moderne, nodigde rapper Duncan uit in de Tweede Kamer en zo ontstond de single Vrijheid, doorregen met samples van Thierry Baudet en zijn nieuwe bestie, Wybre

In [164]:
group['Text2'][0]

"kruiswoordtest 5898; Horizontaal 1deze Europeanen boeken massaal mediterrane vakanties na 17 mei, meldde Trouw vorige week 7 5hoeveel spelers blijven er over als een voetbalelftal waarvan de oudste drie 27 zijn, buiten gaat sporten? 4 10voor welk horlogemerk maakt Roger Federer voor miljoenen euro's per jaar reclame? 5 11zeerovers leiders van een partij die zeggen dat ze socialer zijn dan de SP en liberaler dan de VVD 7 12godin van de wraak of, figuurlijk, de wrekende gerechtigheid 7, anagram van nee, miss 14minister aan wie Nederland de Mammoetwet dankt 4 16verbeterde of geactualiseerde versie leenwoord uit het Engels, 6 18couplet, strofe achtregelig, met rijmschema abababcc 6, anagr. Ans zat 20..... et avoir, Franse film uit 2002 over een dorpsschooltje 4 21onderzoek van 't parlement, naar 't toeslagenschandaal bijvoorbeeld 7 24begraafplaats taalkundige die op YouTube zeventiende-eeuws Hollands laat horen 7 25taal van de band Altin G n die de oprichter ervan zelf niet verstaat 5 27l

In [159]:
#Get cases where there is disagreement amongst coders


# Group the DataFrame by 'unit_id' and 'topic_match' to count the majority votes.
majority_votes = pivoted_df.groupby(['unit_id', 'topic_match']).size().reset_index(name='count')

# Filter for cases where the majority vote is 'Yes,' 'No,' or 'Not sure/Flag.'
majority_agreed_cases = majority_votes[
    (majority_votes['topic_match'] == 'Yes') | 
    (majority_votes['topic_match'] == 'No') | 
    (majority_votes['topic_match'] == 'Not sure/Flag')
]

# Extract 'unit_id' values for the cases where the majority agreed.
majority_agreed_unit_ids = majority_agreed_cases['unit_id'].tolist()

# Filter the original DataFrame to get the rows for the majority agreed cases.
majority_agreed_df = pivoted_df[pivoted_df['unit_id'].isin(majority_agreed_unit_ids)]



In [160]:
majority_agreed_df

,coder_id,coder,jobset,unit_id,unit_status,confidence_1,confidence_2,event_match,topic_match
0,2592,m.simon@uva.nl,All,3285233_3285386,DONE,+/-,++,No,Not sure/Flag
1,2592,m.simon@uva.nl,All,3285240_6290579,DONE,+,++,No,Yes
2,2592,m.simon@uva.nl,All,3285316_3285335,DONE,+,++,No,No
3,2592,m.simon@uva.nl,All,3285325_3285322,DONE,++,++,No,Yes
4,2592,m.simon@uva.nl,All,3285325_3285332,DONE,++,++,No,No
...,...,...,...,...,...,...,...,...,...
351,2604,floris@annotator.com,All,3290679_3287473,DONE,++,++,No,No
352,2604,floris@annotator.com,All,3290679_3290570,DONE,++,++,No,No
353,2604,floris@annotator.com,All,3290697_3287479,DONE,++,++,No,Yes
354,2604,floris@annotator.com,All,3290705_3287467,DONE,++,++,Yes,Yes


import pandas as pd

#Assuming you have your data in 'merged_df' after merging pivoted_df and d
#Replace 'merged_df' with your actual DataFrame

#Calculate majority consensus for each unit_id
majority_consensus = merged_df.groupby('unit_id')['event_match'].apply(lambda x: x.mode().iloc[0]).reset_index()

#Calculate coder disagreement ratio
coder_disagreement = merged_df.groupby(['unit_id', 'coder_id'])['event_match'].apply(lambda x: (x != x.mode().iloc[0]).mean()).reset_index()

#Find the coder with the highest disagreement ratio
coder_with_highest_disagreement = coder_disagreement.groupby('coder_id')['event_match'].mean().idxmax()

print(f"The coder with the highest disagreement ratio is {coder_with_highest_disagreement}")
